In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import time

import CustomHyperModel
import CustomWindowGenerator
import EnergyPricesLibrary as Ep

from kerastuner.tuners import BayesianOptimization

%load_ext autoreload
%autoreload 2

# DATASET

## Load Dataset

In [2]:
data_diaria_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Diaria.xlsx')
data_diaria = pd.read_excel(data_diaria_path)
data_diaria = data_diaria.set_index('Fecha')

In [3]:
nombre_series = data_diaria.columns.values

In [4]:
precio_bolsa_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Precio_Bolsa.xlsx')
precio_bolsa = pd.read_excel(precio_bolsa_path)
precio_bolsa = precio_bolsa.set_index('Fecha')

In [5]:
data_diaria.shape, precio_bolsa.shape

((7365, 119), (176760, 1))

## Build Window

In [6]:
d = 'All'
time_split = '2020-01-01'
n_steps_in = 1
n_steps_out=24
overlap = 1
inputs_columns = nombre_series
output_columns = ['$kWh']

len_input_features = len(inputs_columns)
len_output_features = len(output_columns)

trainX, trainY, testX, testY, scaler_x,scaler_y, dataset_x, dataset_y = Ep.SplitTimeseriesMultipleTimesBackAhead_differentTimes(
    df_x=data_diaria,
    df_y=precio_bolsa,
    day = d, 
    TimeSplit = time_split,
    n_steps_out=n_steps_out,
    n_steps_in =n_steps_in, 
    overlap = overlap,
    input_features=inputs_columns,
    output_features=output_columns)

trainX.shape,trainY.shape,testX.shape,testY.shape

((7273, 1, 119), (7273, 24, 1), (91, 1, 119), (91, 24, 1))

In [7]:
"""
log_dir = os.path.join('logs','Optimización')
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
callback_tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                          histogram_freq=2,
                                                          write_graph=False,
                                                          update_freq='epoch
"""
    
callback_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                          factor=0.1,
                                                          min_lr=1e-4,
                                                          patience=0,
                                                          verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=5,
                                                  mode='min')

#callbacks = [callback_tensorboard,callback_reduce_lr,early_stopping]
callbacks = [callback_reduce_lr,early_stopping]

In [8]:
INPUT_SHAPE = (n_steps_in,len_input_features)

arquitectura31 = CustomHyperModel.Arquitectura31(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura32 = CustomHyperModel.Arquitectura32(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura33 = CustomHyperModel.Arquitectura33(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura34 = CustomHyperModel.Arquitectura34(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura35 = CustomHyperModel.Arquitectura35(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura36 = CustomHyperModel.Arquitectura36(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura37 = CustomHyperModel.Arquitectura37(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura38 = CustomHyperModel.Arquitectura38(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)

arquitectura39 = CustomHyperModel.Arquitectura39(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura40 = CustomHyperModel.Arquitectura40(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura41 = CustomHyperModel.Arquitectura41(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura42 = CustomHyperModel.Arquitectura42(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura43 = CustomHyperModel.Arquitectura43(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura44 = CustomHyperModel.Arquitectura44(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura45 = CustomHyperModel.Arquitectura45(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura46 = CustomHyperModel.Arquitectura46(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)

In [9]:
arq_list = [arquitectura31,arquitectura32,arquitectura33,arquitectura34,
            arquitectura35,arquitectura36,arquitectura37,arquitectura38,
            arquitectura39,arquitectura40,arquitectura41,arquitectura42,
            arquitectura43,arquitectura44,arquitectura45,arquitectura46]
"""
arq_list = [arquitectura35,arquitectura36,arquitectura37,arquitectura38,
            arquitectura39,arquitectura40,arquitectura41,arquitectura42,
            arquitectura43,arquitectura44,arquitectura45,arquitectura46]
"""

'\narq_list = [arquitectura35,arquitectura36,arquitectura37,arquitectura38,\n            arquitectura39,arquitectura40,arquitectura41,arquitectura42,\n            arquitectura43,arquitectura44,arquitectura45,arquitectura46]\n'

In [10]:
arq_idx = 31
arq_best_models = dict()

for arq in arq_list:
    
    bayesian_tuner = BayesianOptimization(
        arq,
        objective='val_mean_absolute_percentage_error',
        num_initial_points=1,
        max_trials=15,
        directory='my_dir',
        project_name=str(arq_idx)
    )
    
    # Overview of the task
    bayesian_tuner.search_space_summary()
    
    # Performs the hyperparameter tuning
    search_start = time.time()
    bayesian_tuner.search(x=trainX,y=trainY,
                      epochs=200,
                      validation_data=(testX,testY),
                      callbacks=callbacks)
    search_end = time.time()
    elapsed_time = search_end - search_start
    
    print('Tiempo Total Transcurrido {}'.format(elapsed_time))
        
    dict_key = 'Arquitectura {}'.format(arq_idx)

    arq_best_models[dict_key] = dict()
    bs_model = bayesian_tuner.oracle.get_best_trials(1)[0]

    arq_best_models[dict_key]['Score'] = bs_model.score
    arq_best_models[dict_key]['Tiempo Scaneo'] = elapsed_time

    if bs_model.hyperparameters.values:
        for hp, value in bs_model.hyperparameters.values.items():
            arq_best_models[dict_key][hp] = value
    
    arq_idx += 1

Trial 15 Complete [00h 00m 44s]
val_mean_absolute_percentage_error: 21.736282348632812

Best val_mean_absolute_percentage_error So Far: 21.736282348632812
Total elapsed time: 00h 05m 36s
INFO:tensorflow:Oracle triggered exit
Tiempo Total Transcurrido 336.1096291542053


In [11]:
import json
with open('BestModels.json', 'w') as outfile:
    json.dump(arq_best_models, outfile)

In [12]:
arq_best_models

{'Arquitectura 31': {'Score': 53.71327590942383,
  'Tiempo Scaneo': 498.51460909843445,
  'lstm_units_layer_1': 512,
  'kernel_regularizer_layer_1': 0.105,
  'dropout_regularizer_layer_1': 0.99,
  'lstm_units_layer_2': 64,
  'kernel_regularizer_layer_2': 0.09,
  'dropout_regularizer_layer_2': 0.0,
  'dense_layer': 120,
  'dense_layer_activation': 'sigmoid',
  'dropout_dense': 0.0,
  'learning_rate': 0.01},
 'Arquitectura 32': {'Score': 59.2144775390625,
  'Tiempo Scaneo': 439.74560356140137,
  'gru_units_layer_1': 192,
  'kernel_regularizer_layer_1': 0.09,
  'dropout_regularizer_layer_1': 0.27,
  'gru_units_layer_2': 64,
  'kernel_regularizer_layer_2': 0.06,
  'dropout_regularizer_layer_2': 0.8999999999999999,
  'dense_layer': 120,
  'dense_layer_activation': 'tanh',
  'dropout_dense': 0.54,
  'learning_rate': 0.009957183217161016},
 'Arquitectura 33': {'Score': 62.037109375,
  'Tiempo Scaneo': 378.313036441803,
  'lstm_units_layer_1': 64,
  'kernel_regularizer_layer_1': 0.105,
  'drop

In [ ]:
bayesian_tuner = BayesianOptimization(
    arquitectura38,
    objective='val_mean_absolute_percentage_error',
    num_initial_points=1,
    max_trials=15,
    directory='my_dir',
    project_name='tuning_38'
)

# Overview of the task
bayesian_tuner.search_space_summary()

# Performs the hyperparameter tuning
search_start = time.time()
bayesian_tuner.search(x=trainX,y=trainY,
                      epochs=200,
                      validation_data=(testX,testY),
                      callbacks=callbacks)
search_end = time.time()
elapsed_time = search_end - search_start

In [ ]:
print('Tiempo Total Transcurrido {}'.format(elapsed_time))

# Show a summary of the search
bayesian_tuner.results_summary()

In [ ]:
# Retrieve the best model.
best_model = bayesian_tuner.get_best_models(num_models=1)[0]

# Evaluate the best model.
loss, mae, mape, smape = best_model.evaluate(x=testX,y=testY)

In [ ]:
best_model.to_json()